# III. Semantic Search on the Vector Database

## Objective
Perform semantic search on the vector database to retrieve the most relevant documents for a given query.

### Approach: [ISODS-PhapDien-Crawler-Semantic-Search](https://github.com/saladnga/ISODS-PhapDien-Crawler-Semantic-Search)

In [3]:
%pip install langchain_chroma sentence_transformers pyvi

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from pyvi import ViTokenizer
import re
import json

/Users/UserN/Desktop/crawling_test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
vector_db_dir = "chroma_db"

### Query Processing:
- Segment the query using ViTokenizer for Vietnamese text
- Replace informal phrases with formal ones using the custom slang.json file for improved accuracy, also remove any punctuations

In [6]:
# Define the Embedding Function
class Embedding:
    def __init__(self, model, slang="slang.json"):
        self.model = model
        self.slang = self.load_slang(slang)

    def load_slang(self, file):
        with open(file, "r", encoding="utf-8") as f:
            return json.load(f)

    def preprocess_text(self, text):
        text = re.sub(r"[^\w\s]", "", text)
        for slang, formal in self.slang.items():
            text = re.sub(rf"\b{slang}\b", formal, text)
        return text

    def embed_query(self, text):
        updated_query = self.preprocess_text(text)
        segmented_query = ViTokenizer.tokenize(updated_query)
        return self.model.encode(segmented_query).tolist()

### Embedding the Query:
- Use the BKAI Vietnamese Bi Encoder model to generate an embedding for the query

In [7]:
# Initialize the model
model = SentenceTransformer("bkai-foundation-models/vietnamese-bi-encoder")

In [8]:
# Initialize the vector database
vector_store = Chroma(
    persist_directory=vector_db_dir,
    embedding_function=Embedding(model),
)

In [9]:
# Check empty database
num_docs = len(vector_store.get()["ids"])
if num_docs == 0:
    print("No documents found in the database")

### Similarity Search:
- Use LangChain’s similarity search to find the most relevant documents in the vector database
- Sort the results by relevance score in descending order

In [10]:
# Semantic search function, top k value
def semantic_search(query, k):
    segmented_query = vector_store._embedding_function.embed_query(query)
    results = vector_store.similarity_search_by_vector_with_relevance_scores(
        segmented_query, k=k
    )

    def get_relevance_score(item):
        return item[1]

    return sorted(results, key=get_relevance_score, reverse=True)

### Return Top-k Results:
- Display the top-k results sorted by their relevance scores

In [11]:
# Execution
query = "chính sách phát triển y tế của nhà nước"
k = 5
results = semantic_search(query, k)

print(f"TOP {k} RESULTS FOR QUERY -'{query}':")
for i, (text, _) in enumerate(results):
    print("--------------------------------------------------------------------------")
    print(f"RESULT NO. {i + 1}: ")
    print("--------------------------------------------------------------------------")
    print(f"METADATA (FILE PATH): {text.metadata}")
    print(f"CONTENT:\n {text.page_content}")
    print("--------------------------------------------------------------------------")


TOP 5 RESULTS FOR QUERY -'chính sách phát triển y tế của nhà nước':
--------------------------------------------------------------------------
RESULT NO. 1: 
--------------------------------------------------------------------------
METADATA (FILE PATH): {'file_path': 'BoPhapDienDienTu/vbpl/full_25170.html'}
CONTENT:
 1. Ưu tiên bố trí ngân sách nhằm đáp ứng nhu cầu khám bệnh, chữa bệnh cơ bản của nhân dân. Quan tâm dành ngân sách cho việc chăm sóc sức khỏe đối với người có công với cách mạng, trẻ em, người nghèo, nông dân, đồng bào dân tộc thiểu số, nhân dân ở vùng có điều kiện kinh tế - xã hội khó khăn và vùng có điều kiện kinh tế - xã hội đặc biệt khó khăn. 
 
 2. Tăng cường phát triển nguồn nhân lực y tế, đặc biệt là nguồn nhân lực y tế ở vùng có điều kiện kinh tế - xã hội khó khăn và vùng có điều kiện kinh tế - xã hội đặc biệt khó khăn. Thực hiện chế độ luân phiên có thời hạn đối với người hành nghề tại cơ sở khám bệnh, chữa bệnh từ tuyến trên xuống tuyến dưới, từ vùng có điều kiệ